# Document-level access example using the push document APIs

In Azure AI Search, you can upload any JSON document payload to a search index for indexing. This notebook shows you how index documents that contain [user access permissions at the document level](azure/search/search-document-level-access-overview), and then query the index to return only those results that the user is authorized to view.

The security principal behind the query access token determines the "user". The permission metadata in the document determines whether the user has authorization to the content. Internally, the search engine filters out any documents that aren't associated with the security principal.

This feature is currently in preview.

For an alternative approaching using indexers and pull API, see [Quickstart-Document-Permissions-Pull-API](../Quickstart-Document-Permissions-Pull-API/document-permissions-pull-api.ipynb).


## Prerequisites

+ Azure AI Search, with [role-based access control](https://learn.microsoft.com/azure/search/search-security-enable-roles).

## Permissions

This walkthrough uses Microsoft Entra ID authentication and authorization.

On Azure AI Search, you must have role assignments to create objects and run queries:

+ **Search Service Contributor**
+ **Search Index Data Contributor**
+ **Search Index Data Reader**

For more information, see [Connect to Azure AI Search using roles](https://learn.microsoft.com/azure/search/search-security-rbac) and [Quickstart: Connect without keys for local testing](https://learn.microsoft.com/azure/search/search-get-started-rbac).

## Set the environment variables

1. Rename `sample.env` to `.env`.
1. In the `.env` file, provide a full endpoint to your search service (https://your-search-service.search.windows.net).
1. Replace the default index name if you want a different name.

## Load Connections

We recommend creating a virtual environment to run this sample code. In Visual Studio Code, open the control palette (ctrl-shift-p) to create an environment. This notebook was tested on Python 3.10.

Once your environment is created, load the environment variables.

In [ ]:
%pip install msgraph-core


In [2]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import os

load_dotenv(override=True) # take environment variables from .env.

# The following variables from your .env file are used in this notebook
endpoint = os.environ["AZURE_SEARCH_ENDPOINT"]
credential = DefaultAzureCredential()
index_name = os.getenv("AZURE_SEARCH_INDEX")
token_provider = get_bearer_token_provider(credential, "https://search.azure.com/.default")


In [8]:
from msgraph.core import GraphClient

# Create the Graph client using Azure Identity credential
graph_client = GraphClient(credential=credential)

# Get current user info
me = graph_client.get('/me').json()
oid = me.get('id')
print(f'User OID: {oid}')

# Get group memberships
groups_response = graph_client.get('/me/memberOf?$select=id')
groups = groups_response.json().get('value', [])
group_ids = [g['id'] for g in groups if 'id' in g]
print(f'Group IDs: {group_ids}')

ModuleNotFoundError: No module named 'msgraph.core'

## Create Sample Index

The search index must includes fields for your content and for permission metadata. Assign the new permission filter option to a string field and make sure the field is filterable. The search engine builds the filter internally at query time.

In [3]:
from azure.search.documents.indexes.models import SearchField, SearchIndex, PermissionFilter, SearchIndexPermissionFilterOption
from azure.search.documents.indexes import SearchIndexClient

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index = SearchIndex(
    name=index_name,
    fields=[
        SearchField(name="id", type="Edm.String", key=True, filterable=True, sortable=True),
        SearchField(name="oid", type="Collection(Edm.String)", retrievable=True, filterable=True, permission_filter=PermissionFilter.USER_IDS),
        SearchField(name="group", type="Collection(Edm.String)", retrievable=True, filterable=True, permission_filter=PermissionFilter.GROUP_IDS),
        SearchField(name="name", type="Edm.String", searchable=True)
    ],
    permission_filter_option=SearchIndexPermissionFilterOption.ENABLED
)

index_client.create_index(index=index)
print(f"Index '{index_name}' created with permission filter option enabled.")

retrievable is not a known attribute of class <class 'azure.search.documents.indexes.models._index.SearchField'> and will be ignored
retrievable is not a known attribute of class <class 'azure.search.documents.indexes.models._index.SearchField'> and will be ignored


HttpResponseError: (ResourceNameAlreadyInUse) Cannot create index 'document-permissions-push-idx' because it already exists.
Code: ResourceNameAlreadyInUse
Message: Cannot create index 'document-permissions-push-idx' because it already exists.
Exception Details:	(CannotCreateExistingIndex) Cannot create index 'document-permissions-push-idx' because it already exists.
	Code: CannotCreateExistingIndex
	Message: Cannot create index 'document-permissions-push-idx' because it already exists.

## Connect to Graph to find your object ID (OID) and groups

This step calls the Graph APIs to get a few group IDs for your Microsoft Entra identity. Your group IDs will be added to the access control list of the objects created in the next step.

In [4]:
from msgraph.core import GraphSession

print("msgraphcore is installed.")
print("Version:", msgraphcore.__version__)

ModuleNotFoundError: No module named 'msgraph.core'

In [5]:
from azure.search.documents.indexes.models import SearchField, SearchIndex, PermissionFilter, SearchIndexPermissionFilterOption
from azure.search.documents.indexes import SearchIndexClient

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index = SearchIndex(
    name=index_name,
    fields=[
        SearchField(name="id", type="Edm.String", key=True, filterable=True, sortable=True),
        SearchField(name="oid", type="Collection(Edm.String)", retrievable=True, filterable=True, permission_filter=PermissionFilter.USER_IDS),
        SearchField(name="group", type="Collection(Edm.String)", retrievable=True, filterable=True, permission_filter=PermissionFilter.GROUP_IDS),
        SearchField(name="name", type="Edm.String", searchable=True)
    ],
    permission_filter_option=SearchIndexPermissionFilterOption.ENABLED
)

index_client.create_index(index=index)
print(f"Index '{index_name}' created with permission filter option enabled.")

retrievable is not a known attribute of class <class 'azure.search.documents.indexes.models._index.SearchField'> and will be ignored
retrievable is not a known attribute of class <class 'azure.search.documents.indexes.models._index.SearchField'> and will be ignored


HttpResponseError: (ResourceNameAlreadyInUse) Cannot create index 'document-permissions-push-idx' because it already exists.
Code: ResourceNameAlreadyInUse
Message: Cannot create index 'document-permissions-push-idx' because it already exists.
Exception Details:	(CannotCreateExistingIndex) Cannot create index 'document-permissions-push-idx' because it already exists.
	Code: CannotCreateExistingIndex
	Message: Cannot create index 'document-permissions-push-idx' because it already exists.

In [6]:
# Install msgraph-core if you haven't already
from msgraph.core import GraphClient

# Create the Graph client using Azure Identity credential
graph_client = GraphClient(credential=credential)

# Get current user info
me = graph_client.get('/me').json()
oid = me.get('id')
print(f'User OID: {oid}')

# Get group memberships
groups_response = graph_client.get('/me/memberOf?$select=id')
groups = groups_response.json().get('value', [])
group_ids = [g['id'] for g in groups if 'id' in g]
print(f'Group IDs: {group_ids}')

ModuleNotFoundError: No module named 'msgraph.core'

In [ ]:
from azure.search.documents.indexes.models import SearchField, SearchIndex, PermissionFilter, SearchIndexPermissionFilterOption
from azure.search.documents.indexes import SearchIndexClient

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index = SearchIndex(
    name=index_name,
    fields=[
        SearchField(name="id", type="Edm.String", key=True, filterable=True, sortable=True),
        SearchField(name="oid", type="Collection(Edm.String)", retrievable=True, filterable=True, permission_filter=PermissionFilter.USER_IDS),
        SearchField(name="group", type="Collection(Edm.String)", retrievable=True, filterable=True, permission_filter=PermissionFilter.GROUP_IDS),
        SearchField(name="name", type="Edm.String", searchable=True)
    ],
    permission_filter_option=SearchIndexPermissionFilterOption.ENABLED
)

index_client.create_index(index=index)
print(f"Index '{index_name}' created with permission filter option enabled.")

In [ ]:
from azure.search.documents.indexes.models import SearchField, SearchIndex, PermissionFilter, SearchIndexPermissionFilterOption
from azure.search.documents.indexes import SearchIndexClient

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index = SearchIndex(
    name=index_name,
    fields=[
        SearchField(name="id", type="Edm.String", key=True, filterable=True, sortable=True),
        SearchField(name="oid", type="Collection(Edm.String)", retrievable=True, filterable=True, permission_filter=PermissionFilter.USER_IDS),
        SearchField(name="group", type="Collection(Edm.String)", retrievable=True, filterable=True, permission_filter=PermissionFilter.GROUP_IDS),
        SearchField(name="name", type="Edm.String", searchable=True)
    ],
    permission_filter_option=SearchIndexPermissionFilterOption.ENABLED
)

index_client.create_index(index=index)
print(f"Index '{index_name}' created with permission filter option enabled.")

## Upload Sample Data

This step creates the container, folders, and uploads documents into Azure Storage. It assigns your group IDs to to the access control list for each file.

In [ ]:
from azure.search.documents import SearchClient
search_client = SearchClient(endpoint=endpoint, index_name=index_name, credential=credential)

documents = [
    { "id": "1", "oid": [oid], "group": [groups.value[0].id], "name": "Document 1" },
    { "id": "2", "oid": ["all"], "group": [groups.value[0].id], "name": "Document 2" },
    { "id": "3", "oid": [oid], "group": ["all"], "name": "Document 3" },
    { "id": "4", "oid": ["none"], "group": ["none"], "name": "Document 4" },
    { "id": "5", "oid": ["none"], "group": [groups.value[0].id], "name": "Document 5" },
]
search_client.upload_documents(documents=documents)
print("Documents uploaded to the index.")


## Search sample data with x-ms-query-source-authorization

This query uses an empty search string (`*`) to provide an unqualified search. It returns the file name and permission metadata associated with each file. Notice that each file is associated with a different group ID.

In [ ]:
results = search_client.search(search_text="*", x_ms_query_source_authorization=token_provider(), select="name,oid,group", order_by="id asc")

for result in results:
    print(f"Name: {result['name']}, OID: {result['oid']}, Group: {result['group']}")

## Search sample data without x-ms-query-source-authorization 

This step demonstrates the user experience when authorization fails. No results are returned in the response.

In [ ]:
results = search_client.search(search_text="*", x_ms_query_source_authorization=None, select="name,oid,group", order_by="id asc")

for result in results:
    print(f"Name: {result['name']}, OID: {result['oid']}, Group: {result['group']}")

## Next steps

To learn more, see [Document-level access control in Azure AI Search](https://learn.microsoft.com/azure/search/search-document-level-access-overview).